In [1]:
import numpy as np
import gym
from gym import Env, spaces, utils
import pygame
from typing import Optional
from io import StringIO
from contextlib import closing

In [2]:
from env_2048 import *

In [5]:
env = BlockDoubleEnv()
env.reset()  # reset enviornment to default state
action = env.action_space.sample()  # get a random action 
new_state, reward, done, info = env.step(action)  # take action, notice it returns information about the action
print(env.render())   # render the GUI for the enviornment 

[0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]
[2.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0]



In [7]:
STATES = env.observation_space.n
ACTIONS = env.action_space.n
Q = np.zeros((STATES, ACTIONS))  # create a matrix with all 0 values 
Q
https://stackoverflow.com/questions/66892382/q-table-with-gym-using-box-observation-space

AttributeError: 'Box' object has no attribute 'n'

In [ ]:
EPISODES = 2000 # how many times to run the enviornment from the beginning
MAX_STEPS = 100  # max number of steps allowed for each run of enviornment

LEARNING_RATE = 0.81  # learning rate
GAMMA = 0.96